In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import optimizers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
import time
from tqdm import tqdm_notebook, tqdm
from natsort import natsorted
from collections import deque
import joblib
import colorama as col

In [2]:
x = 'EUR/USD'
_period = 'm5'
_period2 = 'H1'
_ticker = x.replace('/','')
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2016-06-30' # stop the train and begin the test there '2016-08-31'
_stop = '2017-12-31' # stop the test there. After that, it is kept for oos '2017-12-31'
_last = '2021-04-29' # '2020-12-31'

def sigmoid(x):
  return 1 - (1 + math.exp(-x))

def stock_price_format(n):
  if n < 0:
    return "- # {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [3]:
def dataset_loader():

    data = joblib.load('BASES/EURUSD_m5')

    start_date = str(data.index[0]).split()[0]
    end_date = str(data.index[1]).split()[0]

    _sl = 0.001
    _target = 0.002

    # scaler = Normalizer()
    scaler = MinMaxScaler(feature_range=(-1, 1))
    # scaler = StandardScaler()

    def strategy(df):
        ##### CONDITIONS LONG
        _condition_1 = (df.slow_K5 < 20) & (df.slow_K5.shift(1) < df.slow_D5.shift(1)) & (df.slow_K5 > df.slow_D5)

        ##### CONDITIONS SHORT
        _condition_1_bar = (df.slow_K5 > 80) & (df.slow_K5.shift(1) > df.slow_D5.shift(1)) & (df.slow_K5 < df.slow_D5)

        ##### 1 condition
        df['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))
        df = df.drop(['Symbol','Date','DateIndex','SB_Gamma'], axis=1)
        return(df.sort_index(axis=0)) 

    def strategy5(df,_window=40):

        #df['RSI_2'] = TA.RSI(ohlc=df,int=2,str='Close')

        df['Window_High_Ask'] = df.HighAsk.iloc[::-1].rolling(_window).max().iloc[::-1] # Limite SL Short
        df['Window_High_Bid'] = df.HighBid.iloc[::-1].rolling(_window).max().iloc[::-1] # Limite Target Long
        df['Window_Low_Ask'] = df.LowAsk.iloc[::-1].rolling(_window).min().iloc[::-1] # Limite Target Short
        df['Window_Low_Bid'] = df.LowBid.iloc[::-1].rolling(_window).min().iloc[::-1] # Limite SL Long
        df['Window_sl_Short'] = df.CloseBid + (df.CloseBid * _sl) # Short pour SL
        df['Window_sl_Long'] = df.CloseAsk - (df.CloseAsk * _sl) # Long pour SL
        df['Window_tp_Short'] = df.CloseBid - (df.CloseBid * _target) # Short pour TP
        df['Window_tp_Long'] = df.CloseAsk + (df.CloseAsk * _target) # Long pour TP

        ##### CONDITIONS LONG
        _condition_1 = (df['Window_tp_Long'] <= df['Window_High_Bid']) & (df['Window_sl_Long'] <= df['Window_Low_Bid'])

        ##### CONDITIONS SHORT
        _condition_1_bar = (df['Window_tp_Short'] >= df['Window_Low_Ask']) & (df['Window_sl_Short'] >= df['Window_High_Ask'])

        ##### 1 condition
        df['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))
        df = df.drop(['Symbol','Date','DateIndex','Window_High_Ask','Window_High_Bid','Window_Low_Ask','Window_Low_Bid','Window_sl_Short','Window_sl_Long','Window_tp_Short','Window_tp_Long'], axis=1)
        return(df.sort_index(axis=0))


    data = klines(data)

    data = strategy(data)

    data = data[['Body','Color','UpperShadow','LowerShadow','Signal']]

    data['Body1'] = data['Body'] - data['Body'].shift(1)
    data['Body2'] = data['Body'] - data['Body'].shift(2)
    data['Body3'] = data['Body'] - data['Body'].shift(3)
    data['Body4'] = data['Body'] - data['Body'].shift(4)
    data['Body5'] = data['Body'] - data['Body'].shift(5)

    data['UpperShadow1'] = data['UpperShadow'] - data['UpperShadow'].shift(1)
    data['UpperShadow2'] = data['UpperShadow'] - data['UpperShadow'].shift(2)
    data['UpperShadow3'] = data['UpperShadow'] - data['UpperShadow'].shift(3)
    data['UpperShadow4'] = data['UpperShadow'] - data['UpperShadow'].shift(4)
    data['UpperShadow5'] = data['UpperShadow'] - data['UpperShadow'].shift(5)

    data['LowerShadow1'] = data['LowerShadow'] - data['LowerShadow'].shift(1)
    data['LowerShadow2'] = data['LowerShadow'] - data['LowerShadow'].shift(2)
    data['LowerShadow3'] = data['LowerShadow'] - data['LowerShadow'].shift(3)
    data['LowerShadow4'] = data['LowerShadow'] - data['LowerShadow'].shift(4)
    data['LowerShadow5'] = data['LowerShadow'] - data['LowerShadow'].shift(5)


    df_train, df_test, df_oos = split_df(data)

    df_oos_raw = df_oos.copy().dropna()

    df_train.sort_index(inplace=True)
    
    for i in df_train.columns.unique():
        if i != 'Signal' and i != 'Color':
            df_train[i] = scaler.fit_transform(df_train[i].values.reshape(-1, 1))
            df_test[i] = scaler.fit_transform(df_test[i].values.reshape(-1, 1))
            df_oos[i] = scaler.fit_transform(df_oos[i].values.reshape(-1, 1))

    df_train = df_train.dropna()
    df_test = df_test.dropna()
    df_oos = df_oos.dropna()

    signal_train = df_train['Signal']
    signal_test = df_test['Signal']
    signal_oos = df_oos['Signal']

    df_train = df_train.drop(['Signal'],axis=1)
    df_test = df_test.drop(['Signal'],axis=1)
    df_oos = df_oos.drop(['Signal'],axis=1)

    df_oos = df_oos.reindex(natsorted(df_oos.columns), axis=1)
    df_test = df_test.reindex(natsorted(df_test.columns), axis=1)
    df_train = df_train.reindex(natsorted(df_train.columns), axis=1)
    df_oos_raw = df_oos_raw.reindex(natsorted(df_oos_raw.columns), axis=1)

    return df_train, df_test, df_oos, df_oos_raw, signal_train, signal_test, signal_oos

def state_creator(data, timestep): #, window_size):

  state = []
  state.append(data.iloc[timestep,:])
    
  return np.array([state])

In [4]:
def klines(df):
    _condition1 = df.Close >= df.Open
    df['Color'] = np.where(_condition1,1,-1)
    _condition2 = df.Color = 1
    df['UpperShadow'] = np.where(_condition2,(df.High-df.Close),(df.High-df.Open))
    df['LowerShadow'] = np.where(_condition2,(df.Open-df.Low),(df.Close-df.Low))
    df['Body'] = abs(df.Close-df.Open)
    return (df)

In [5]:
def split_df(df):
    """[Split the dtaframe in train/test/oos and reduce train and test to dataframe with signal!=0 only]

    Args:
        df ([pandas]): [the datframe to split already featured]
        _start ([date]): [beginning of the df]
        _mid ([date]): [stop of the train and beginning of the test]
        _stop ([date]): [stop of the test and beginning of the oos]
        _last ([date]): [end of the oos]
    """    
    df = df.dropna()
    df['Date'] = pd.to_datetime(df.index)
    
    df_train = df[(df.Date>=_start)&(df.Date<=_mid)]
    df_test = df[(df.Date>_mid)&(df.Date<=_stop)]
    df_oos = df[(df.Date>_stop)&(df.Date <= _last)]
    df_train = df_train.drop(['Date'],axis=1)
    df_test = df_test.drop(['Date'],axis=1)
    df_oos = df_oos.drop(['Date'],axis=1)
    return(df_train, df_test,df_oos)

In [6]:
%%time
df_train, df_test, df_oos, df_oos_raw, signal_train, signal_test, signal_oos = dataset_loader()

<ipython-input-5-0658ba4c6534>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df.index)
CPU times: user 1.39 s, sys: 842 ms, total: 2.23 s
Wall time: 2.1 s


In [7]:
window_size = 10
episodes = 1000
state_size = df_train.shape[1]
batch_size = 50000
data_samples = 500 #len(df_train) - 1

In [8]:
class AI_Trader():
  
    def __init__(self, state_size=state_size, action_space=3, model_name="AITrader"):

        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque(maxlen=5000)
        self.inventory = []
        self.model_name = model_name
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995
        self.model = self.model_builder()

    def model_builder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Input(df_train.shape[1]))
        model.add(tf.keras.layers.Dense(units=64, activation='relu')) # , input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        model.add(tf.keras.layers.Dense(units=3, activation='tanh'))
        #model = tf.model({inputs: input, outputs: dense2})
        model.output_shape
        model.compile(optimizers.Adam(lr=0.001),loss='mse')
        return model  

    

    def trade(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
        
        actions = self.model.predict(state)
        return actions

    def batch_train(self, batch_size):

        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])

            for state, action, reward, next_state, done in batch:
                reward = reward
            
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target = self.model.predict(state)
            target[0][action] = reward

            self.model.fit(state, target, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

        
#return model



In [9]:
trader = AI_Trader(df_train,window_size,state_size)
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1280      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 18,051
Trainable params: 18,051
Non-trainable params: 0
_________________________________________________________________


In [10]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  # state = state_creator(df_train, 0)
  
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    state = df_train.iloc[t:t+1,:]
    # next_state = state_creator(data, t+1, window_size + 1)
    next_state = df_train.iloc[t+1:t+2,:]
    action = trader.trade(state)
    reward = 0
    
    if action == 1 and signal_train[t] == 1: #Buying
      reward = 1
      total_profit += reward
      
    elif action == 2 and signal_train[t] == -1: #Selling
      reward = 1
      total_profit += reward
    
    elif action == 0 and signal_train[t] == 0: #Selling
      reward = 0
      total_profit += reward
    
    elif action == 0 and signal_train[t] != 0: #Selling
      reward = -1
      total_profit += reward
    
    elif action != 0 and signal_train[t] == 0: #Selling
      reward = -1
      total_profit += reward
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
        time.sleep(0.2)
        print("\n"+col.Fore.CYAN+"########################"+col.Fore.GREEN)
        print("TOTAL PROFIT:  {} ".format(total_profit))
        print(col.Fore.CYAN+"########################"+col.Style.RESET_ALL)
        print()
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

 29%|██▉       | 146/500 [00:00<00:00, 1333.58it/s]
########################
TOTAL PROFIT:  -425 
########################

Episode: 2/1000
 93%|█████████▎| 464/500 [00:00<00:00, 4626.35it/s]
########################
TOTAL PROFIT:  -433 
########################

Episode: 3/1000
 93%|█████████▎| 467/500 [00:00<00:00, 4666.30it/s]
########################
TOTAL PROFIT:  -433 
########################

Episode: 4/1000
 93%|█████████▎| 466/500 [00:00<00:00, 4657.15it/s]
########################
TOTAL PROFIT:  -430 
########################

Episode: 5/1000
 81%|████████▏ | 407/500 [00:00<00:00, 2366.91it/s]
########################
TOTAL PROFIT:  -427 
########################

Episode: 6/1000
 89%|████████▉ | 445/500 [00:00<00:00, 4448.53it/s]
########################
TOTAL PROFIT:  -443 
########################

Episode: 7/1000
 88%|████████▊ | 438/500 [00:00<00:00, 4373.92it/s]
########################
TOTAL PROFIT:  -421 
########################

Episode: 8/1000
 92%|█████████▏| 460

KeyboardInterrupt: 

2

### VERIFICATION DU PREDICT

In [11]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(df_train.shape[1]))
model.add(tf.keras.layers.Dense(units=64, activation='relu')) # , input_dim=self.state_size))
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dense(units=3, activation='relu'))
#model = tf.model({inputs: input, outputs: dense2})
model.output_shape
model.compile(optimizers.Adam(lr=0.001),loss='mse')


In [24]:
np.array([state])

array([[[-8.07986406e-01, -4.24607155e-02,  7.20850086e-02,
          7.17761557e-02,  9.38559010e-02,  7.33825445e-02,
          1.00000000e+00, -9.95541401e-01, -9.68655923e-02,
         -5.92381272e-02, -5.84250635e-02,  2.26923733e-02,
         -3.06564479e-02, -9.61700747e-01, -9.58855098e-02,
         -2.90933694e-02,  3.27332242e-04,  1.52162763e-02,
         -5.42367007e-02]]])

### VERIFICATION DE LA GENERATION DES ACTIONS

In [24]:
trader.trade(state)

4

### VERIFICAYION BATCH

In [25]:
batch = []
memory = df_train.iloc[:5000,:]
for i in range(len(memory) - batch_size + 1, len(memory)):
    batch.append(memory[i])

    for state, action, reward, next_state, done in batch:
        reward = reward
    
    if not done:
        reward = reward + gamma * np.amax(model.predict(next_state)[0])

    target = model.predict(state)
    target[0][action] = reward

    model.fit(state, target, epochs=1, verbose=0)

if epsilon > epsilon_final:
    epsilon *= epsilon_decay

NameError: name 'memory' is not defined

### VERIFICATION INTEGRITE DES DATA => CORRESPONDANCE DF ET SIGNAL

In [12]:
data = joblib.load('BASES/EURUSD_m5')

start_date = str(data.index[0]).split()[0]
end_date = str(data.index[1]).split()[0]

_sl = 0.001
_target = 0.002

# scaler = Normalizer()
scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler = StandardScaler()

def strategy(df):
    ##### CONDITIONS LONG
    _condition_1 = (df.slow_K5 < 20) & (df.slow_K5.shift(1) < df.slow_D5.shift(1)) & (df.slow_K5 > df.slow_D5)

    ##### CONDITIONS SHORT
    _condition_1_bar = (df.slow_K5 > 80) & (df.slow_K5.shift(1) > df.slow_D5.shift(1)) & (df.slow_K5 < df.slow_D5)

    ##### 1 condition
    df['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))
    df = df.drop(['Symbol','Date','DateIndex','SB_Gamma'], axis=1)
    return(df.sort_index(axis=0)) 

def strategy5(df,_window=40):

    #df['RSI_2'] = TA.RSI(ohlc=df,int=2,str='Close')

    df['Window_High_Ask'] = df.HighAsk.iloc[::-1].rolling(_window).max().iloc[::-1] # Limite SL Short
    df['Window_High_Bid'] = df.HighBid.iloc[::-1].rolling(_window).max().iloc[::-1] # Limite Target Long
    df['Window_Low_Ask'] = df.LowAsk.iloc[::-1].rolling(_window).min().iloc[::-1] # Limite Target Short
    df['Window_Low_Bid'] = df.LowBid.iloc[::-1].rolling(_window).min().iloc[::-1] # Limite SL Long
    df['Window_sl_Short'] = df.CloseBid + (df.CloseBid * _sl) # Short pour SL
    df['Window_sl_Long'] = df.CloseAsk - (df.CloseAsk * _sl) # Long pour SL
    df['Window_tp_Short'] = df.CloseBid - (df.CloseBid * _target) # Short pour TP
    df['Window_tp_Long'] = df.CloseAsk + (df.CloseAsk * _target) # Long pour TP

    ##### CONDITIONS LONG
    _condition_1 = (df['Window_tp_Long'] <= df['Window_High_Bid']) & (df['Window_sl_Long'] <= df['Window_Low_Bid'])

    ##### CONDITIONS SHORT
    _condition_1_bar = (df['Window_tp_Short'] >= df['Window_Low_Ask']) & (df['Window_sl_Short'] >= df['Window_High_Ask'])

    ##### 1 condition
    df['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))
    df = df.drop(['Symbol','Date','DateIndex','Window_High_Ask','Window_High_Bid','Window_Low_Ask','Window_Low_Bid','Window_sl_Short','Window_sl_Long','Window_tp_Short','Window_tp_Long'], axis=1)
    return(df.sort_index(axis=0))


data = klines(data)

data = strategy(data)

data = data[['Body','Color','UpperShadow','LowerShadow','Signal']]

data['Body1'] = data['Body'] - data['Body'].shift(1)
data['Body2'] = data['Body'] - data['Body'].shift(2)
data['Body3'] = data['Body'] - data['Body'].shift(3)
data['Body4'] = data['Body'] - data['Body'].shift(4)
data['Body5'] = data['Body'] - data['Body'].shift(5)

data['UpperShadow1'] = data['UpperShadow'] - data['UpperShadow'].shift(1)
data['UpperShadow2'] = data['UpperShadow'] - data['UpperShadow'].shift(2)
data['UpperShadow3'] = data['UpperShadow'] - data['UpperShadow'].shift(3)
data['UpperShadow4'] = data['UpperShadow'] - data['UpperShadow'].shift(4)
data['UpperShadow5'] = data['UpperShadow'] - data['UpperShadow'].shift(5)

data['LowerShadow1'] = data['LowerShadow'] - data['LowerShadow'].shift(1)
data['LowerShadow2'] = data['LowerShadow'] - data['LowerShadow'].shift(2)
data['LowerShadow3'] = data['LowerShadow'] - data['LowerShadow'].shift(3)
data['LowerShadow4'] = data['LowerShadow'] - data['LowerShadow'].shift(4)
data['LowerShadow5'] = data['LowerShadow'] - data['LowerShadow'].shift(5)

data = data.dropna()

df_train, df_test, df_oos = split_df(data)

df_oos_raw = df_oos.copy().dropna()

df_train.sort_index(inplace=True)

for i in df_train.columns.unique():
    if i != 'Signal' and i != 'Color':
        df_train[i] = scaler.fit_transform(df_train[i].values.reshape(-1, 1))
        df_test[i] = scaler.fit_transform(df_test[i].values.reshape(-1, 1))
        df_oos[i] = scaler.fit_transform(df_oos[i].values.reshape(-1, 1))


signal_train = df_train['Signal']
signal_test = df_test['Signal']
signal_oos = df_oos['Signal']

df_train = df_train.drop(['Signal'],axis=1)
df_test = df_test.drop(['Signal'],axis=1)
df_oos = df_oos.drop(['Signal'],axis=1)

df_oos = df_oos.reindex(natsorted(df_oos.columns), axis=1)
df_test = df_test.reindex(natsorted(df_test.columns), axis=1)
df_train = df_train.reindex(natsorted(df_train.columns), axis=1)
df_oos_raw = df_oos_raw.reindex(natsorted(df_oos_raw.columns), axis=1)

In [13]:
df_train.iloc[46:47,:]

,Body,Body1,Body2,Body3,Body4,Body5,Color,LowerShadow,LowerShadow1,LowerShadow2,LowerShadow3,LowerShadow4,LowerShadow5,UpperShadow,UpperShadow1,UpperShadow2,UpperShadow3,UpperShadow4,UpperShadow5
2010-01-04 04:15:00,-0.960918,-0.07556,-0.005744,-0.010341,0.004273,0.004487,1,-0.978981,-0.107845,-0.026682,-0.048264,0.014844,-0.031656,-0.998702,-0.106977,-0.019959,-0.002619,0.019661,-0.039335


In [14]:
signal_train.iloc[46] # 8, 16, 22, 46

-1

In [15]:
data.iloc[46:47,:].Signal 

2010-01-04 04:15:00   -1
Name: Signal, dtype: int64